In [1]:
import os
os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

Current working directory is C:\Users\Gubbz\Documents\NSS\NSS_Projects\accre-pumpkin-pie


In [2]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [3]:
jobs = pd.read_csv("data/fullsample.csv")
jobs.head(5)

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0


In [4]:
jobs_filtered = jobs[(jobs['STATE'] == 'COMPLETED') | (jobs['STATE'] == 'FAILED')].copy()

In [5]:
jobs_filtered['USEDMEM'] = jobs_filtered['USEDMEM'].str.replace('M','').astype(float)
jobs_filtered

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06T16:17:34,16384Mn,9708.04,24-09:00:00,20-08:32:27,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
7395880,25493434,COMPLETED,2020-10-31T23:39:00,2020-10-31T23:40:46,2000Mn,0.09,2-00:00:00,00:01:46,1,1,sam,0:0
7395881,25493435,COMPLETED,2020-10-31T23:39:13,2020-10-31T23:40:38,2000Mn,187.92,2-00:00:00,00:01:25,1,1,sam,0:0
7395882,25493476,COMPLETED,2020-10-31T23:46:29,2020-10-31T23:49:43,4096Mc,803.97,12:00:00,00:03:14,1,1,production,0:0
7395883,25493515,COMPLETED,2020-10-31T23:49:44,2020-10-31T23:51:40,2000Mn,0.09,2-00:00:00,00:01:56,1,1,sam,0:0


## Investigating Issues between Node and Cores

<h4>Per Core

In [8]:
jobs_mem_per_core = jobs[jobs['REQMEM'].astype(str).str.endswith('c')]
jobs_mem_per_core

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
501,31776583_1,COMPLETED,2021-08-30T10:16:59,2021-09-01T02:04:11,4096Mc,1792.43M,14-00:00:00,1-15:47:12,1,1,production,0:0
502,31776584_12,COMPLETED,2021-08-30T10:17:00,2021-09-01T00:20:15,4096Mc,1792.43M,14-00:00:00,1-14:03:15,1,1,production,0:0
915,31793401_958,COMPLETED,2021-08-31T19:36:46,2021-09-01T00:37:11,4096Mc,2788.05M,05:00:00,05:00:25,1,1,production,0:0
916,31793401_987,COMPLETED,2021-08-31T20:33:46,2021-09-01T00:02:57,4096Mc,2779.27M,05:00:00,03:29:11,1,1,production,0:0
4727,31813223_1296,COMPLETED,2021-08-31T19:42:46,2021-09-01T00:43:15,4096Mc,2786.44M,05:00:00,05:00:29,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
7395874,25493206,COMPLETED,2020-10-31T23:18:27,2020-10-31T23:23:31,4096Mc,852.30M,12:00:00,00:05:04,1,1,production,0:0
7395877,25493287,COMPLETED,2020-10-31T23:27:04,2020-10-31T23:30:16,4096Mc,801.25M,12:00:00,00:03:12,1,1,production,0:0
7395878,25493390,COMPLETED,2020-10-31T23:35:40,2020-10-31T23:39:12,4096Mc,813.09M,12:00:00,00:03:32,1,1,production,0:0
7395882,25493476,COMPLETED,2020-10-31T23:46:29,2020-10-31T23:49:43,4096Mc,803.97M,12:00:00,00:03:14,1,1,production,0:0


In [9]:
jobs_mem_per_core['REQMEM'] = jobs_mem_per_core['REQMEM'].str.replace('Mc','').astype(float)

C:\Users\Gubbz\AppData\Local\Temp\ipykernel_89228\3592037547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_mem_per_core['REQMEM'] = jobs_mem_per_core['REQMEM'].str.replace('Mc','').astype(float)


In [22]:
jobs_mem_per_core['TOTAL_MEM_REQ'] = jobs_mem_per_core['REQMEM']*jobs_mem_per_core['CPUS']
jobs_mem_per_core.head(10)

C:\Users\Gubbz\AppData\Local\Temp\ipykernel_89228\1705742728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_mem_per_core['TOTAL_MEM_REQ'] = jobs_mem_per_core['REQMEM']*jobs_mem_per_core['CPUS']


,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,TOTAL_MEM_REQ
501,31776583_1,COMPLETED,2021-08-30T10:16:59,2021-09-01T02:04:11,4096.0,1792.43M,14-00:00:00,1-15:47:12,1,1,production,0:0,4096.0
502,31776584_12,COMPLETED,2021-08-30T10:17:00,2021-09-01T00:20:15,4096.0,1792.43M,14-00:00:00,1-14:03:15,1,1,production,0:0,4096.0
915,31793401_958,COMPLETED,2021-08-31T19:36:46,2021-09-01T00:37:11,4096.0,2788.05M,05:00:00,05:00:25,1,1,production,0:0,4096.0
916,31793401_987,COMPLETED,2021-08-31T20:33:46,2021-09-01T00:02:57,4096.0,2779.27M,05:00:00,03:29:11,1,1,production,0:0,4096.0
4727,31813223_1296,COMPLETED,2021-08-31T19:42:46,2021-09-01T00:43:15,4096.0,2786.44M,05:00:00,05:00:29,1,1,production,0:0,4096.0
4728,31813223_1338,COMPLETED,2021-08-31T20:46:46,2021-09-01T00:12:10,4096.0,2782.86M,05:00:00,03:25:24,1,1,production,0:0,4096.0
4729,31813223_1350,COMPLETED,2021-08-31T21:00:46,2021-09-01T02:00:51,4096.0,2778.21M,05:00:00,05:00:05,1,1,production,0:0,4096.0
4730,31813223_1354,COMPLETED,2021-08-31T21:20:21,2021-09-01T00:03:38,4096.0,2784.43M,05:00:00,02:43:17,1,1,production,0:0,4096.0
4731,31813223_1355,COMPLETED,2021-08-31T21:20:21,2021-09-01T00:07:12,4096.0,2784.07M,05:00:00,02:46:51,1,1,production,0:0,4096.0
4732,31813223_1356,COMPLETED,2021-08-31T21:20:21,2021-09-01T00:02:14,4096.0,2790.02M,05:00:00,02:41:53,1,1,production,0:0,4096.0


<h4>Per Node

In [12]:
jobs_mem_per_node = jobs[jobs['REQMEM'].astype(str).str.endswith('n')]
jobs_mem_per_node

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
7395876,25493268,COMPLETED,2020-10-31T23:27:04,2020-10-31T23:38:46,5000Mn,736.60M,2-00:00:00,00:11:42,1,1,production,0:0
7395879,25493400,COMPLETED,2020-10-31T23:34:23,2020-10-31T23:35:39,2000Mn,0.09M,2-00:00:00,00:01:16,1,1,sam,0:0
7395880,25493434,COMPLETED,2020-10-31T23:39:00,2020-10-31T23:40:46,2000Mn,0.09M,2-00:00:00,00:01:46,1,1,sam,0:0
7395881,25493435,COMPLETED,2020-10-31T23:39:13,2020-10-31T23:40:38,2000Mn,187.92M,2-00:00:00,00:01:25,1,1,sam,0:0


In [13]:
jobs_mem_per_node['REQMEM'] = jobs_mem_per_node['REQMEM'].str.replace('Mn','').astype(float)

C:\Users\Gubbz\AppData\Local\Temp\ipykernel_89228\3915032830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_mem_per_node['REQMEM'] = jobs_mem_per_node['REQMEM'].str.replace('Mn','').astype(float)


In [14]:
jobs_mem_per_node['TOTAL_MEM_REQ'] = jobs_mem_per_node['REQMEM']*jobs_mem_per_node['NODES']
jobs_mem_per_node

C:\Users\Gubbz\AppData\Local\Temp\ipykernel_89228\3742971186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_mem_per_node['TOTAL_MEM_REQ'] = jobs_mem_per_node['REQMEM']*jobs_mem_per_node['NODES']


,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,TOTAL_MEM_REQ
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048.0,0,10:04:00,67-22:14:22,1,1,production,0:0,2048.0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144.0,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0,262144.0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800.0,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0,204800.0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536.0,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0,65536.0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384.0,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0,16384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7395876,25493268,COMPLETED,2020-10-31T23:27:04,2020-10-31T23:38:46,5000.0,736.60M,2-00:00:00,00:11:42,1,1,production,0:0,5000.0
7395879,25493400,COMPLETED,2020-10-31T23:34:23,2020-10-31T23:35:39,2000.0,0.09M,2-00:00:00,00:01:16,1,1,sam,0:0,2000.0
7395880,25493434,COMPLETED,2020-10-31T23:39:00,2020-10-31T23:40:46,2000.0,0.09M,2-00:00:00,00:01:46,1,1,sam,0:0,2000.0
7395881,25493435,COMPLETED,2020-10-31T23:39:13,2020-10-31T23:40:38,2000.0,187.92M,2-00:00:00,00:01:25,1,1,sam,0:0,2000.0


## Memory Difference

In [16]:
# jobs_filtered['REQMEM'] = jobs_filtered['REQMEM'].str.replace('Mn','').str.replace('Mc','').astype(float)

In [17]:
# jobs_filtered[jobs_filtered['USEDMEM']>jobs_filtered['REQMEM']]

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
422,31772052,COMPLETED,2021-08-30T17:08:17,2021-09-01T17:08:47,5000.0,5178.03,2-00:00:00,2-00:00:30,1,1,production,0:0
850,31793023,COMPLETED,2021-08-30T21:05:23,2021-09-01T01:22:19,5000.0,5019.74,2-00:00:00,1-04:16:56,1,1,production,0:0
25261,31878761,COMPLETED,2021-08-31T21:28:40,2021-09-04T14:59:37,204800.0,205378.32,4-00:00:00,3-17:30:57,1,100,production,0:0
26514,31928065_6,COMPLETED,2021-09-01T02:42:57,2021-09-01T02:52:37,16384.0,16771.78,04:00:00,00:09:40,1,1,production,0:0
26529,31928065_21,COMPLETED,2021-09-01T02:43:50,2021-09-01T02:54:32,16384.0,16687.11,04:00:00,00:10:42,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
7391387,25479426,COMPLETED,2020-10-31T02:47:34,2020-10-31T03:56:42,5000.0,5111.20,2-00:00:00,01:09:08,1,1,production,0:0
7391406,25479641,COMPLETED,2020-10-31T03:15:33,2020-10-31T03:40:11,5000.0,5285.77,2-00:00:00,00:24:38,1,1,production,0:0
7391512,25480724,COMPLETED,2020-10-31T05:29:26,2020-10-31T07:58:49,5000.0,5299.10,2-00:00:00,02:29:23,1,1,production,0:0
7392652,25485057,COMPLETED,2020-10-31T10:30:05,2020-10-31T11:24:43,5000.0,5020.54,2-00:00:00,00:54:38,1,1,production,0:0


In [24]:
# jobs_filtered['MEM_Diff'] = jobs_filtered['REQMEM'] - jobs_filtered['USEDMEM']

In [19]:
# jobs_filtered['Over_Used_MEM'] = jobs_filtered['MEM_Diff'].apply(lambda x: 0 if x > 0 else 1)

In [26]:
# jobs_filtered

In [28]:
# jobs_mem_per_core = jobs_filtered[jobs_filtered['REQMEM']]